Parameter settings

In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [2]:
from numpy import log, e
import pint
from scipy.constants import Stefan_Boltzmann # W m^-2 K^-4
ureg = pint.UnitRegistry()

In [12]:
#  ∆RF = 5.35ln(C/Co)
# 405
# 315
# https://journals.sagepub.com/doi/pdf/10.1260/0958-305X.25.8.1439#:~:text=On%20this%20basis%2C%20the%20IPCC,to%20the%20Beer%2DLambert%20law.
# IPCC relation
5.35*log(405/315) / log(e) * ureg.watt / ureg.meter ** 2 

1.3445321913028478 <Unit('watt / meter ** 2')>

In [18]:
def ipcc_delta_co2(ratio: float):
    return 5.35*log(ratio) / log(e) * ureg.watt / ureg.meter ** 2

print(ipcc_delta_co2(2))
print(ipcc_delta_co2(400/300))

3.708337415995707 watt / meter ** 2
1.5390990876170274 watt / meter ** 2


In [24]:
log(1.33) * 3

0.8555368267009874

In [ ]:
# Lightfoot and Mamer 2014 relation
def rf_quad_co2(conc: float):
    return (-0.00002480* conc**2 + 0.03231 * conc) * ureg.watt / ureg.meter ** 2

rf_quad_co2(405) - rf_quad_co2(315)

## redoing f1prime calculation in the paper

In [27]:
# c. 300 to c. 400 ppm
def f1_prime(
    delta_forcing=0.75 * ureg.watt / ureg.meter ** 2,  # W m -2
    temp_surface=301 * ureg.kelvin,  # K
    cloud_cover=0.6,  # dimensionless fraction
    cloud_constant_a=0.8,  # dimensionless fraction
    emissivity=0.97,  # dimensionless fraction
):
    stefan_boltzmann = (
        Stefan_Boltzmann * ureg.watt / ureg.meter ** 2 / ureg.kelvin ** 4
    )  # W m^-2 K^-4
    return -delta_forcing / (
        emissivity
        * stefan_boltzmann
        * (temp_surface ** 4)
        * (1 - cloud_constant_a * (cloud_cover ** 2))
    )

In [28]:
print(f1_prime())

-0.002333091742004311 dimensionless


In [30]:
print(f1_prime(delta_forcing=rf_quad_co2(405) - rf_quad_co2(315)))

-0.004046700964671638 dimensionless


In [31]:
print(f1_prime(delta_forcing=ipcc_delta_co2(400/300)))

-0.004787812495260875 dimensionless


In [32]:
print(f1_prime(delta_forcing=ipcc_delta_co2(405/315), cloud_constant_a=0.6))

-0.0037984436566326457 dimensionless


In [6]:
temp_surface.to(ureg.celsius)
# almost 28 degrees

27.850000000000023 <Unit('degree_Celsius')>

In [25]:
type(temp_surface)

pint.quantity.build_quantity_class.<locals>.Quantity